# 빅데이터분석기사 실기 대비
예제 문제 풀이: 자세한 설명 [링크](https://djccnt15.github.io/dataanalysis/bigdata_certi_01/)

문제 출처: [제2회 빅데이터분석기사 실기 안내(수정: 체험링크 추가)](https://www.dataq.or.kr/www/board/view.do?bbsKey=eyJiYnNhdHRyU2VxIjoxLCJiYnNTZXEiOjUwOTM0M30=&boardKind=notice)

## 작업형 제1유형 : 데이터 처리 영역

mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소최대 척도(Min-Max Scale)로 변환한 후 0.5보다 큰 값을 가지는 레코드 수를 구하시오.

In [1]:
import pandas as pd

df = pd.read_csv('data/mtcars.csv')

In [2]:
from sklearn.preprocessing import MinMaxScaler

data = df[['qsec']]

scaler = MinMaxScaler()
scaler.fit(data)
scaled = scaler.transform(data)

print(len(scaled[scaled > 0.5]))

9


## 작업형 제2유형 : 모형 구축 및 평가 영역

고객 3,500명에 대한 학습용 데이터(`y_train.csv`, `X_train.csv`)를 이용하여 성별예측 모형을 만든 후, 이를 평가용 데이터(`X_test.csv`)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 `CSV` 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

**제출형식**

```
custid,gender
3500,0.267
3501,0.578
3502,0.885
```

❗ 1회차 시험 때 제출 형식의 칼럼명을 다르게 썼다고 **0점 처리**된 사람이 많다는 얘기가 있다. 제출 형식에 주의해야 한다.

**유의사항**

성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다.

### 데이터 읽기

In [3]:
import pandas as pd

exog = pd.read_csv('data/X_train.csv', encoding='euc-kr')
endog = pd.read_csv('data/y_train.csv', encoding='euc-kr')
test = pd.read_csv('data/X_test.csv', encoding='euc-kr')

### EDA 진행

In [4]:
print(exog.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
None


In [5]:
print(endog.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   cust_id  3500 non-null   int64
 1   gender   3500 non-null   int64
dtypes: int64(2)
memory usage: 54.8 KB
None


In [6]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  2482 non-null   int64  
 1   총구매액     2482 non-null   int64  
 2   최대구매액    2482 non-null   int64  
 3   환불금액     871 non-null    float64
 4   주구매상품    2482 non-null   object 
 5   주구매지점    2482 non-null   object 
 6   내점일수     2482 non-null   int64  
 7   내점당구매건수  2482 non-null   float64
 8   주말방문비율   2482 non-null   float64
 9   구매주기     2482 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 194.0+ KB
None


In [7]:
print(pd.unique(exog['주구매상품']))
print(pd.unique(test['주구매상품']))
print(len(pd.unique(exog['주구매상품'])), len(pd.unique(test['주구매상품'])))

['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']
['골프' '농산물' '가공식품' '주방용품' '수산품' '화장품' '기타' '스포츠' '디자이너' '시티웨어' '구두' '캐주얼'
 '명품' '건강식품' '남성정장' '커리어' '남성 캐주얼' '축산가공' '식기' '피혁잡화' '모피/피혁' '섬유잡화'
 '트래디셔널' '차/커피' '육류' '가구' '아동' '셔츠' '액세서리' '젓갈/반찬' '대형가전' '일용잡화' '통신/컴퓨터'
 '생활잡화' '주방가전' '란제리/내의' '남성 트랜디' '보석' '주류' '침구/수예' '악기']
42 41


In [8]:
print(pd.unique(exog['주구매지점']))
print(pd.unique(test['주구매지점']))
print(len(pd.unique(exog['주구매지점'])), len(pd.unique(test['주구매지점'])))

['강남점' '잠실점' '관악점' '광주점' '본  점' '일산점' '대전점' '부산본점' '분당점' '영등포점' '미아점'
 '청량리점' '안양점' '부평점' '동래점' '포항점' '노원점' '창원점' '센텀시티점' '인천점' '대구점' '전주점'
 '울산점' '상인점']
['부산본점' '잠실점' '본  점' '청량리점' '분당점' '일산점' '대전점' '강남점' '동래점' '영등포점' '부평점'
 '대구점' '노원점' '광주점' '관악점' '미아점' '창원점' '인천점' '안양점' '상인점' '포항점' '울산점' '전주점'
 '센텀시티점']
24 24


In [9]:
obj_cols = exog.select_dtypes(include='object').columns

print(obj_cols)

Index(['주구매상품', '주구매지점'], dtype='object')


In [10]:
num_cols = [i for i in exog.columns if i not in obj_cols]
cust_id = test.loc[:,'cust_id']
num_cols.remove('cust_id')

print(num_cols)

['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']


### Table join

In [11]:
df = pd.concat([exog, test])

In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5982 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  5982 non-null   int64  
 1   총구매액     5982 non-null   int64  
 2   최대구매액    5982 non-null   int64  
 3   환불금액     2076 non-null   float64
 4   주구매상품    5982 non-null   object 
 5   주구매지점    5982 non-null   object 
 6   내점일수     5982 non-null   int64  
 7   내점당구매건수  5982 non-null   float64
 8   주말방문비율   5982 non-null   float64
 9   구매주기     5982 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 514.1+ KB
None


### 필요 없는 칼럼 제거

In [13]:
df.drop(columns=['cust_id'], inplace=True)
endog.drop(columns=['cust_id'], inplace=True)

### 결측치 제거

In [14]:
df['환불금액'].fillna(value=0, inplace=True)

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5982 entries, 0 to 2481
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     5982 non-null   int64  
 1   최대구매액    5982 non-null   int64  
 2   환불금액     5982 non-null   float64
 3   주구매상품    5982 non-null   object 
 4   주구매지점    5982 non-null   object 
 5   내점일수     5982 non-null   int64  
 6   내점당구매건수  5982 non-null   float64
 7   주말방문비율   5982 non-null   float64
 8   구매주기     5982 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 467.3+ KB
None


### 정규화

In [16]:
import numpy as np
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
for col in num_cols:
    df[col] = scaler.fit_transform(np.array(df[col]).reshape(-1, 1))

### One-hot encoding

In [17]:
encoded = pd.get_dummies(df[['주구매상품', '주구매지점']])
df = pd.concat([df, encoded], axis=1)
df.drop(columns=['주구매상품', '주구매지점'], inplace=True)

In [18]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5982 entries, 0 to 2481
Data columns (total 73 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   총구매액          5982 non-null   float64
 1   최대구매액         5982 non-null   float64
 2   환불금액          5982 non-null   float64
 3   내점일수          5982 non-null   float64
 4   내점당구매건수       5982 non-null   float64
 5   주말방문비율        5982 non-null   float64
 6   구매주기          5982 non-null   float64
 7   주구매상품_가공식품    5982 non-null   uint8  
 8   주구매상품_가구      5982 non-null   uint8  
 9   주구매상품_건강식품    5982 non-null   uint8  
 10  주구매상품_골프      5982 non-null   uint8  
 11  주구매상품_구두      5982 non-null   uint8  
 12  주구매상품_기타      5982 non-null   uint8  
 13  주구매상품_남성 캐주얼  5982 non-null   uint8  
 14  주구매상품_남성 트랜디  5982 non-null   uint8  
 15  주구매상품_남성정장    5982 non-null   uint8  
 16  주구매상품_농산물     5982 non-null   uint8  
 17  주구매상품_대형가전    5982 non-null   uint8  
 18  주구매상품_디자이너    5982 non-null 

### Table split

In [19]:
exog = df.iloc[:3500, :]
test = df.iloc[3500:, :]

In [20]:
print(exog.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 3499
Data columns (total 73 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   총구매액          3500 non-null   float64
 1   최대구매액         3500 non-null   float64
 2   환불금액          3500 non-null   float64
 3   내점일수          3500 non-null   float64
 4   내점당구매건수       3500 non-null   float64
 5   주말방문비율        3500 non-null   float64
 6   구매주기          3500 non-null   float64
 7   주구매상품_가공식품    3500 non-null   uint8  
 8   주구매상품_가구      3500 non-null   uint8  
 9   주구매상품_건강식품    3500 non-null   uint8  
 10  주구매상품_골프      3500 non-null   uint8  
 11  주구매상품_구두      3500 non-null   uint8  
 12  주구매상품_기타      3500 non-null   uint8  
 13  주구매상품_남성 캐주얼  3500 non-null   uint8  
 14  주구매상품_남성 트랜디  3500 non-null   uint8  
 15  주구매상품_남성정장    3500 non-null   uint8  
 16  주구매상품_농산물     3500 non-null   uint8  
 17  주구매상품_대형가전    3500 non-null   uint8  
 18  주구매상품_디자이너    3500 non-null 

In [21]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2482 entries, 0 to 2481
Data columns (total 73 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   총구매액          2482 non-null   float64
 1   최대구매액         2482 non-null   float64
 2   환불금액          2482 non-null   float64
 3   내점일수          2482 non-null   float64
 4   내점당구매건수       2482 non-null   float64
 5   주말방문비율        2482 non-null   float64
 6   구매주기          2482 non-null   float64
 7   주구매상품_가공식품    2482 non-null   uint8  
 8   주구매상품_가구      2482 non-null   uint8  
 9   주구매상품_건강식품    2482 non-null   uint8  
 10  주구매상품_골프      2482 non-null   uint8  
 11  주구매상품_구두      2482 non-null   uint8  
 12  주구매상품_기타      2482 non-null   uint8  
 13  주구매상품_남성 캐주얼  2482 non-null   uint8  
 14  주구매상품_남성 트랜디  2482 non-null   uint8  
 15  주구매상품_남성정장    2482 non-null   uint8  
 16  주구매상품_농산물     2482 non-null   uint8  
 17  주구매상품_대형가전    2482 non-null   uint8  
 18  주구매상품_디자이너    2482 non-null 

In [23]:
print(endog.info())

<class 'pandas.core.series.Series'>
RangeIndex: 3500 entries, 0 to 3499
Series name: gender
Non-Null Count  Dtype
--------------  -----
3500 non-null   int64
dtypes: int64(1)
memory usage: 27.5 KB
None


In [22]:
endog = endog['gender']

### Model 생성 및 학습

In [24]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(exog, endog)

c:\projects\certi\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
predict = model.predict_proba(test)

In [26]:
print(predict)

[[0.44712369 0.55287631]
 [0.86261603 0.13738397]
 [0.81802694 0.18197306]
 ...
 [0.25974833 0.74025167]
 [0.53228477 0.46771523]
 [0.52846109 0.47153891]]


In [27]:
predict_man = predict[:, 1]

### 제출용 파일 저장

In [28]:
result = pd.DataFrame({'custid':cust_id, 'gender':predict_man})

In [29]:
print(result.head())

   custid    gender
0    3500  0.552876
1    3501  0.137384
2    3502  0.181973
3    3503  0.353272
4    3504  0.436386


In [30]:
result.to_csv('result.csv', index=False)